In [7]:
import numpy as np
import scipy as sp
import pandas as pd
import warnings

import cython
%load_ext Cython
from iminuit import Minuit
idx = pd.IndexSlice

import clapy
import clasim


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# Data for time series and histogramms

In [8]:
asym_dist = clapy.dist()

dargs = {
        'samples': 10000,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.5,
        'S': 0.3,
        'G2M': 0.2,
        'sCells' : 0.3,
        'sSamples' : 0.2
}

dTC = dargs['G1']+dargs['G2M']+dargs['S']
dFS = dargs['S']/dTC
X = np.arange(0,dargs['nCells']+1)
time_points = np.linspace(0.01,1.965625,22)
measure_times = np.ravel(np.array(time_points)[:,np.newaxis]*np.ones(dargs['samples']))


pdfs = list()
pdfm = list()
data_asy = list()
data_sym = list()

for t in time_points:
    #simulations asymetric
    data_asy.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=1,times=[t],**dargs)) )
    #simulations symetric
    data_sym.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=2,times=[t],**dargs)) )
    #pdfs
    pdfs.append( [asym_dist.pmf_f(dargs['nCells'],dTC,dFS,dargs['GF'],dargs['sCells'],dargs['sSamples'],t,i) for i in X] )
    #means
    pdfm.append( asym_dist.pmf_mean(dargs['nCells'],dTC,dFS,dargs['GF'],dargs['sCells'],dargs['sSamples'],t) )

    
with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['data_asy'] = pd.Series(data_asy)
    st['data_sym'] = pd.Series(data_sym)
    st['pdfs']     = pd.Series(pdfs)
    st['pdfm']     = pd.Series(pdfm)


# Gernerate data for parameter recovery

In [9]:
%%cython --force
# distutils: language = c++

#use --annotate if you wonder what kind of code it generates 
cimport cython
import numpy as np
cimport numpy as np #overwritten those from python with cython
from libc.math cimport exp, M_PI, sqrt, log
from iminuit.util import describe, make_func_code
from libcpp.map cimport map     
import scipy as sp



@cython.embedsignature(True)#dump the signatre so describe works
cpdef fitfunc(double t,double Tc,double r,double GF):
    cdef double res = 0
    if t<(Tc-Tc*r):
        res = GF/Tc*(t+Tc*r)
    else:
        res =  GF
    return res 
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdf(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = 1./(sqrt(2*M_PI*std))
    cdef double ret = exp(-1*(x-mu)*(x-mu)/(2.*std))*norm
    return ret
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdfln(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = (sqrt(2*M_PI*std*std))
    cdef double ret = (-1*(x-mu)*(x-mu)/(2.*std*std))-log(norm)
    return ret




cdef class Nowakowski_LH:
    cdef np.ndarray data
    cdef np.ndarray err
    cdef np.ndarray t
    cdef int ndata
    
    def __init__(self,data,t):
        self.data = data
        self.t = t
        self.ndata = len(data)
    
    @cython.embedsignature(True)#you need this to dump function signature in docstring
    def compute(self, double Tc,double r,double GF,double s):
        #this line is a cast not a copy. Let cython knows mydata will spit out double
        cdef np.ndarray[np.double_t, ndim=1] mydata = self.data
        cdef np.ndarray[np.double_t, ndim=1] myt = self.t
        cdef double loglh = 0.
        cdef double lmu = 0.
        cdef double ler = 0.
        for i in range(self.ndata):
            lmu = fitfunc(myt[i],Tc,r,GF)
            loglh -= mypdfln(mydata[i],lmu,s)
        return loglh



## with 5 samples

In [10]:
default_args = {
        'samples': 5,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.5,
        'S': 0.3,
        'G2M': 0.2,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
times=np.linspace(0.01,1.5,5)
#default_args['samples'] = int(np.round(120/(mm)))


index = pd.MultiIndex.from_product([[],[]],names=['std','index'])
# Priors for unknown model parameters

tmplist = []
tmplist2= []
for num in range(1000):
    Y,X = np.array(clasim.run(seed=2*num,mode=1,times=times,**default_args))
    

    lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
    #lh.compute(0.4,5,3)

    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
               error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
               limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
               errordef=0.5,print_level=0)
    mi_old.migrad(ncall=999999999);
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'old'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])

    tmplist2.append(num)
    tmplist.append(s)



    Y,X = np.array(clasim.run(seed=2*num+1,mode=1,times=times,**default_args))

    lh = clapy.asym_lh(Y,X,100)
    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
       error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
       limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
       errordef=0.5,print_level=0)
    mi_old.migrad();
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'full'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])


    tmplist2.append(num)
    tmplist.append(s)   
    print('finiscged',num,s['model'],"t")

stat5 = pd.DataFrame(tmplist,index=tmplist2)
stat5.rename_axis({"GF":"GFf"}, axis="columns",inplace=True)
stat5.index.rename('N',inplace=True)
stat5['sSamples'] = 0.2
stat5['sCells'] = 0.3
stat5.set_index('sSamples', append=True, inplace=True)
stat5.set_index('sCells', append=True, inplace=True)
stat5.set_index('model', append=True, inplace=True)    

stat5 =  stat5.reorder_levels(['model','sSamples','sCells','N']) 

warnings.filterwarnings('default')

stat5 = stat5.sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['minuit_full_t001_15_s5_GF095_m5'] = stat5

finiscged 0 full t
finiscged 1 full t
finiscged 2 full t
finiscged 3 full t
finiscged 4 full t
finiscged 5 full t
finiscged 6 full t
finiscged 7 full t
finiscged 8 full t
finiscged 9 full t
finiscged 10 full t
finiscged 11 full t
finiscged 12 full t
finiscged 13 full t
finiscged 14 full t
finiscged 15 full t
finiscged 16 full t
finiscged 17 full t
finiscged 18 full t
finiscged 19 full t
finiscged 20 full t
finiscged 21 full t
finiscged 22 full t
finiscged 23 full t
finiscged 24 full t
finiscged 25 full t
finiscged 26 full t
finiscged 27 full t
finiscged 28 full t
finiscged 29 full t
finiscged 30 full t
finiscged 31 full t
finiscged 32 full t
finiscged 33 full t
finiscged 34 full t
finiscged 35 full t
finiscged 36 full t
finiscged 37 full t
finiscged 38 full t
finiscged 39 full t
finiscged 40 full t
finiscged 41 full t
finiscged 42 full t
finiscged 43 full t
finiscged 44 full t
finiscged 45 full t
finiscged 46 full t
finiscged 47 full t
finiscged 48 full t
finiscged 49 full t
finiscged 

finiscged 396 full t
finiscged 397 full t
finiscged 398 full t
finiscged 399 full t
finiscged 400 full t
finiscged 401 full t
finiscged 402 full t
finiscged 403 full t
finiscged 404 full t
finiscged 405 full t
finiscged 406 full t
finiscged 407 full t
finiscged 408 full t
finiscged 409 full t
finiscged 410 full t
finiscged 411 full t
finiscged 412 full t
finiscged 413 full t
finiscged 414 full t
finiscged 415 full t
finiscged 416 full t
finiscged 417 full t
finiscged 418 full t
finiscged 419 full t
finiscged 420 full t
finiscged 421 full t
finiscged 422 full t
finiscged 423 full t
finiscged 424 full t
finiscged 425 full t
finiscged 426 full t
finiscged 427 full t
finiscged 428 full t
finiscged 429 full t
finiscged 430 full t
finiscged 431 full t
finiscged 432 full t
finiscged 433 full t
finiscged 434 full t
finiscged 435 full t
finiscged 436 full t
finiscged 437 full t
finiscged 438 full t
finiscged 439 full t
finiscged 440 full t
finiscged 441 full t
finiscged 442 full t
finiscged 443

finiscged 787 full t
finiscged 788 full t
finiscged 789 full t
finiscged 790 full t
finiscged 791 full t
finiscged 792 full t
finiscged 793 full t
finiscged 794 full t
finiscged 795 full t
finiscged 796 full t
finiscged 797 full t
finiscged 798 full t
finiscged 799 full t
finiscged 800 full t
finiscged 801 full t
finiscged 802 full t
finiscged 803 full t
finiscged 804 full t
finiscged 805 full t
finiscged 806 full t
finiscged 807 full t
finiscged 808 full t
finiscged 809 full t
finiscged 810 full t
finiscged 811 full t
finiscged 812 full t
finiscged 813 full t
finiscged 814 full t
finiscged 815 full t
finiscged 816 full t
finiscged 817 full t
finiscged 818 full t
finiscged 819 full t
finiscged 820 full t
finiscged 821 full t
finiscged 822 full t
finiscged 823 full t
finiscged 824 full t
finiscged 825 full t
finiscged 826 full t
finiscged 827 full t
finiscged 828 full t
finiscged 829 full t
finiscged 830 full t
finiscged 831 full t
finiscged 832 full t
finiscged 833 full t
finiscged 834

## with 100 samples

In [11]:
default_args = {
        'samples': 100,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.5,
        'S': 0.3,
        'G2M': 0.2,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
times=np.linspace(0.01,1.5,5)
#default_args['samples'] = int(np.round(120/(mm)))


index = pd.MultiIndex.from_product([[],[]],names=['std','index'])
# Priors for unknown model parameters
SM = 456426
tmplist = []
tmplist2= []
for num in range(1000):
    Y,X = np.array(clasim.run(seed=num*2+SM,mode=1,times=times,**default_args))

    lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
    #lh.compute(0.4,5,3)

    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
               error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
               limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
               errordef=0.5,print_level=0)
    mi_old.migrad(ncall=999999999);
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'old'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])

    tmplist2.append(num)
    tmplist.append(s)



    Y,X = np.array(clasim.run(seed=num*2+SM+1,mode=1,times=times,**default_args))


    lh = clapy.asym_lh(Y,X,100)
    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
       error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
       limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
       errordef=0.5,print_level=0)
    mi_old.migrad();
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'full'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])


    tmplist2.append(num)
    tmplist.append(s)   
    print('finiscged',num,s['model'],"t")

stat100 = pd.DataFrame(tmplist,index=tmplist2)
stat100.rename_axis({"GF":"GFf"}, axis="columns",inplace=True)
stat100.index.rename('N',inplace=True)
stat100['sSamples'] = 0.2
stat100['sCells'] = 0.3
stat100.set_index('sSamples', append=True, inplace=True)
stat100.set_index('sCells', append=True, inplace=True)
stat100.set_index('model', append=True, inplace=True)    

stat100 =  stat100.reorder_levels(['model','sSamples','sCells','N']) 

warnings.filterwarnings('default')

stat100 = stat100.sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['minuit_full_t001_15_s100_GF095_m5'] = stat100

finiscged 0 full t
finiscged 1 full t
finiscged 2 full t
finiscged 3 full t
finiscged 4 full t
finiscged 5 full t
finiscged 6 full t
finiscged 7 full t
finiscged 8 full t
finiscged 9 full t
finiscged 10 full t
finiscged 11 full t
finiscged 12 full t
finiscged 13 full t
finiscged 14 full t
finiscged 15 full t
finiscged 16 full t
finiscged 17 full t
finiscged 18 full t
finiscged 19 full t
finiscged 20 full t
finiscged 21 full t
finiscged 22 full t
finiscged 23 full t
finiscged 24 full t
finiscged 25 full t
finiscged 26 full t
finiscged 27 full t
finiscged 28 full t
finiscged 29 full t
finiscged 30 full t
finiscged 31 full t
finiscged 32 full t
finiscged 33 full t
finiscged 34 full t
finiscged 35 full t
finiscged 36 full t
finiscged 37 full t
finiscged 38 full t
finiscged 39 full t
finiscged 40 full t
finiscged 41 full t
finiscged 42 full t
finiscged 43 full t
finiscged 44 full t
finiscged 45 full t
finiscged 46 full t
finiscged 47 full t
finiscged 48 full t
finiscged 49 full t
finiscged 

finiscged 396 full t
finiscged 397 full t
finiscged 398 full t
finiscged 399 full t
finiscged 400 full t
finiscged 401 full t
finiscged 402 full t
finiscged 403 full t
finiscged 404 full t
finiscged 405 full t
finiscged 406 full t
finiscged 407 full t
finiscged 408 full t
finiscged 409 full t
finiscged 410 full t
finiscged 411 full t
finiscged 412 full t
finiscged 413 full t
finiscged 414 full t
finiscged 415 full t
finiscged 416 full t
finiscged 417 full t
finiscged 418 full t
finiscged 419 full t
finiscged 420 full t
finiscged 421 full t
finiscged 422 full t
finiscged 423 full t
finiscged 424 full t
finiscged 425 full t
finiscged 426 full t
finiscged 427 full t
finiscged 428 full t
finiscged 429 full t
finiscged 430 full t
finiscged 431 full t
finiscged 432 full t
finiscged 433 full t
finiscged 434 full t
finiscged 435 full t
finiscged 436 full t
finiscged 437 full t
finiscged 438 full t
finiscged 439 full t
finiscged 440 full t
finiscged 441 full t
finiscged 442 full t
finiscged 443

finiscged 787 full t
finiscged 788 full t
finiscged 789 full t
finiscged 790 full t
finiscged 791 full t
finiscged 792 full t
finiscged 793 full t
finiscged 794 full t
finiscged 795 full t
finiscged 796 full t
finiscged 797 full t
finiscged 798 full t
finiscged 799 full t
finiscged 800 full t
finiscged 801 full t
finiscged 802 full t
finiscged 803 full t
finiscged 804 full t
finiscged 805 full t
finiscged 806 full t
finiscged 807 full t
finiscged 808 full t
finiscged 809 full t
finiscged 810 full t
finiscged 811 full t
finiscged 812 full t
finiscged 813 full t
finiscged 814 full t
finiscged 815 full t
finiscged 816 full t
finiscged 817 full t
finiscged 818 full t
finiscged 819 full t
finiscged 820 full t
finiscged 821 full t
finiscged 822 full t
finiscged 823 full t
finiscged 824 full t
finiscged 825 full t
finiscged 826 full t
finiscged 827 full t
finiscged 828 full t
finiscged 829 full t
finiscged 830 full t
finiscged 831 full t
finiscged 832 full t
finiscged 833 full t
finiscged 834

## for different initial conditions

In [11]:
default_args = {
        'samples': 100,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.2,
        'S': 0.3,
        'G2M': 0.5,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
allim = []
sss = 0
SM = 1286321
for start in np.linspace(0.01,0.5,49*1+1): #range(5,31,5):
    for leng in np.linspace(0.5,2,15*1+1):
        for n in range(10):
            times=np.linspace(start,start+leng,5)
            Y,X = np.array(clasim.run(seed=sss+SM,mode=1,times=times,**default_args))
            sss+1
            lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
            

            mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
                       error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
                       limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
                       errordef=0.5,print_level=0)
            mi_old.migrad(ncall=999999999);
            s = dict(mi_old.values)
            for ii in mi_old.errors:
                s.update({ii+"_error" : mi_old.errors[ii]})
            s.update({'N' : n})
            if calc_error:
                try:
                    tmp = mi_old.minos(sigma=2)                    
                    for i in tmp:
                        s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                        s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
                except:
                    print('error in errorfind',s['model'])
            s.update({'leng' : leng})
            s.update({'start' : start})
            nowak = s.copy()



            Y,X = np.array(clasim.run(seed=sss+SM,mode=1,times=times,**default_args))
            sss+1
            
            lh = clapy.asym_lh(Y,X,100)
            mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
               error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
               limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
               errordef=0.5,print_level=0)
            mi_old.migrad();
            s = dict(mi_old.values)
            for ii in mi_old.errors:
                s.update({ii+"_error" : mi_old.errors[ii]})
            s.update({'N' : n})
            s.update({'leng' : leng})
            s.update({'start' : start})

            if calc_error:
                try:
                    tmp = mi_old.minos(sigma=2)                    
                    for i in tmp:
                        s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                        s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
                except:
                    print('error in errorfind',s['model'])


            print('finiscged')

            stat = pd.DataFrame([nowak,s],index=['old','full'])
            stat.set_index('start', append=True, inplace=True)
            stat.set_index('leng', append=True, inplace=True)
            stat.set_index('N', append=True, inplace=True)    

            allim.append( stat )
            #reorder_levels(['start','leng','model']) )

warnings.filterwarnings('default')

allimnew = pd.concat(allim).sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['s100_n10'] = stat100

finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
finiscged
